# Imports

In [22]:
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kurti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Reading reviews 


In [23]:
# Here we are reading our dataset into a Python list. We are also pointing 'open' to the directory we are storing the data from our dataset. For our program, we are storing the dataset into reviews_train and reviews_test
reviews_train = []
for line in open('C:/Users/kurti/Desktop/aclImdb/movie_data/full_train.txt', encoding='utf-8'):
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open('C:/Users/kurti/Desktop/aclImdb/movie_data/full_test.txt', encoding='utf-8'):
    reviews_test.append(line.strip())
    
target = [1 if i < 12500 else 0 for i in range(25000)]

# Cleaning Regex

In [24]:
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)") # Defining the rules for our Regex process. The following symbols listed here will be removed and replaced with a space
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)") # The following expressions listed in here will be replaced with a space in our dataset. 
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews): # Definition of the function we will be using to clean and process our dataset
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews] # The first step of clean and process is to remove the list of symbols in REPLACE_NO_SPACE from our dataset. We are also converting everything to lowercase. 
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews] # Second step of clean and process is to replace the list of expressions in REPLACE_WITH_SPACE with spaces. 
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

# Remove Stop Words (Redacted)

In [25]:
custom_stop_words = ['in', 'of', 'at', 'a', 'the'] 

# To avoid going through the entire English library of Stop Words, we are creating a predefined list of stop words that we will only go through. We will be using this variable as a parameter in our CountVectorizer function. 

# N gram

In [26]:
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 3), stop_words=custom_stop_words) # This is the vectorization process of our program. The 3 parameters used will be explained below.
ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)
# In this process we are converting each review in the dataset into a numeric representation. In our initial base model, we did not implement ngram nor stop words. 
# That would mean that we essentially had a large matrix that has every unique word in our dataset as a column. Then we transformed each review into rows containing 0s and 1s for every matching word in the columns.
# After adding ngrams, we transformed our word feature into trigrams which added more predictive power to our model. We also included a parameter of custom stop words that is defined above. 
# As mentioned before, defining custom stop words will save us time from going through every single stop word in the English library.  

# SVM

In [31]:
X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.75 # We are splitting the training/test set to a 75:25 ratio. Tests that we've done beforehand such as 50:50, 25:75, etc. yielded lower performance. So we decided to use a 75:25 ratio.
)
# Note that we are using the same target here, because the first 12.5k are hte positive reviews while the last 12.5k are the negative reviews. 

# Here we are creating our Classifier. Initially, we had LogisticRegression which was a good baseline model especially because it is a linear method. As discussed during our Final Presentation, Linear Methods work very well on sparse datasets like the one we are using. However,
# For our LogisticRegression model, we only used the parameter c, which determines the regularization of the model. 
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    # lr = LogisticRegression(C=c)
    
    # Here we replaced LogisticRegression with our Support Vector Machine model. LinearSVC worked better because it yielded faster training time with a linear kernel. 
    svm = LinearSVC(C=c, max_iter=5000) # We also added the parameter max_iter to prevent higher c values such as 0.25, 0.5, and 0.1 from going out of range. We were initially receiving errors until we added the max_iter parameter. 
    svm.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, svm.predict(X_val))))
    
    # We are essentially testing which value of c will yield the best accuracy. Once we find the highest value of c, we will then plug that value into our final model to get our final accuracy. 
    # Regularization is used to reduce the risk of overfitting. 

Accuracy for C=0.01: 0.89216
Accuracy for C=0.05: 0.8912
Accuracy for C=0.25: 0.89248
Accuracy for C=0.5: 0.89264
Accuracy for C=1: 0.89264


# Final Accuracy

In [28]:
# After we are able to pinpoint the best value of c, or the strength of regularization, we can compute the final accuracy by inserting the best value of c. 
final_ngram = LinearSVC(C=0.01) # Sicne 0.01 yielded the best result from our test above, we used it as a parameter for LinearSVC to get the final accuracy of 0.90
final_ngram.fit(X, target)
print ("Final Accuracy: %s" 
       % accuracy_score(target, final_ngram.predict(X_test)))

# Final Accuracy with Baseline, no stop words , no ngram, c = 0.05: 0.88144
# Final Accuracy with Stopwords , c = 0.5 = 0.87972
# Final Accuracy with Stop words + Ngram, c = 0.5 = 0.89868
# Final Accuracy with Stop words + Ngram + SVM, c= 0.5 = 0.89412
# Final Accuracy with Stop words + Ngram + SVM, c = 0.01 = 0.89769
# Final Accuracy with custom Stop words + Ngram + SVM, c = 0.01 = 0.9 (BEST VALUE FOR C)

Final Accuracy: 0.9


# Debug Statements 

In [29]:
reviews_train[10]

'When I first read Armistead Maupins story I was taken in by the human drama displayed by Gabriel No one and those he cares about and loves. That being said, we have now been given the film version of an excellent story and are expected to see past the gloss of Hollywood...<br /><br />Writer Armistead Maupin and director Patrick Stettner have truly succeeded! <br /><br />With just the right amount of restraint Robin Williams captures the fragile essence of Gabriel and lets us see his struggle with issues of trust both in his personnel life(Jess) and the world around him(Donna).<br /><br />As we are introduced to the players in this drama we are reminded that nothing is ever as it seems and that the smallest event can change our lives irrevocably. The request to review a book written by a young man turns into a life changing event that helps Gabriel find the strength within himself to carry on and move forward.<br /><br />It\'s to bad that most people will avoid this film. I only say th

In [30]:
reviews_train_clean[10]

'when i first read armistead maupins story i was taken in by the human drama displayed by gabriel no one and those he cares about and loves that being said we have now been given the film version of an excellent story and are expected to see past the gloss of hollywood writer armistead maupin and director patrick stettner have truly succeeded  with just the right amount of restraint robin williams captures the fragile essence of gabriel and lets us see his struggle with issues of trust both in his personnel lifejess and the world around himdonna as we are introduced to the players in this drama we are reminded that nothing is ever as it seems and that the smallest event can change our lives irrevocably the request to review a book written by a young man turns into a life changing event that helps gabriel find the strength within himself to carry on and move forward its to bad that most people will avoid this film i only say that because the average american will probably think robin wi